## HOMEWORK SPARK FOR DIGITAL SKOLA DATA ENGINEER

### STARTING THE SESSION

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("homework") \
    .getOrCreate()

22/11/05 12:31:09 WARN Utils: Your hostname, ALEXA-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.100.207 instead (on interface wlp2s0)
22/11/05 12:31:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/05 12:31:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/05 12:31:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark_session = spark
spark_session

In [4]:
sc = spark_session.sparkContext
sc

<SparkContext master=local[*] appName=homework>

### IMPORT DATA

In [5]:
import os
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
os.getcwd()

'/home/agnes/Documents/digital_skola/Homework/Homework_Agnes_Septilia_Spark'

In [6]:
twitterDF = spark.read. \
    format("csv"). \
    option("inferSchema", "true"). \
    option("header", "true"). \
    load("AkunTwitter_POS.csv")

In [7]:
twitterDF.show()
twitterDF.printSchema()

22/11/05 12:31:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---------------+----------+---------+-------------------+--------------------+---------+------------+-------------+-----+--------------------+-------------------+----+--------------------+-------------+--------------+-----------+--------------------+--------+--------------------+-------+---------+-----+----------+----+----+
|        id|conversation_id|created_at|     date|               time|            timezone|  user_id|    username|         name|place|               tweet|           mentions|urls|              photos|replies_count|retweets_count|likes_count|            hashtags|cashtags|                link|retweet|quote_url|video|user_rt_id|near| geo|
+----------+---------------+----------+---------+-------------------+--------------------+---------+------------+-------------+-----+--------------------+-------------------+----+--------------------+-------------+--------------+-----------+--------------------+--------+--------------------+-------+---------+-----+--------

In [8]:
hashtagDF = spark.read. \
    format("csv"). \
    option("inferSchema", "true"). \
    option("header", "true"). \
    load("HashtagTwitter_POS.csv")

In [9]:
hashtagDF.show()
hashtagDF.printSchema()

+--------+---------------+----------+---------+--------+--------------------+-------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+--------------------+--------+--------------------+-------+---------+-----+----------+----+----+
|      id|conversation_id|created_at|     date|    time|            timezone|      user_id|      username|                name|place|               tweet|            mentions|                urls|              photos|replies_count|retweets_count|likes_count|            hashtags|cashtags|                link|retweet|quote_url|video|user_rt_id|near| geo|
+--------+---------------+----------+---------+--------+--------------------+-------------+--------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+--------------------+--------+--

In [10]:
instaDF = spark.read.format("json").load("Instagram_POS.json")

In [11]:
instaDF.show()
instaDF.printSchema()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|             caption|            comments|            datetime|            img_urls|                 key|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Oranger menjaga s...|[{mamah.asyraf, C...|2019-07-13T04:06:...|[https://scontent...|https://www.insta...|
|Oranger, layanan ...|[{rizqimuhammad77...|2019-07-12T09:55:...|[https://scontent...|https://www.insta...|
|Selamat hari kope...|[{evan_mardiyanto...|2019-07-12T02:21:...|[https://scontent...|https://www.insta...|
|Kuliah? di @polte...|[{dicky.ra_, @pos...|2019-07-11T05:19:...|[https://scontent...|https://www.insta...|
|Selamat ulang tah...|[{hey.rudd, Kak.....|2019-07-11T01:22:...|[https://scontent...|https://www.insta...|
|Kiriman lewat Pos...|[{rienlilitanty, ...|2019-07-10T10:46:...|[https://scontent...|https://www.insta...|
|Sedang mencari ka...|[{nurdianing, m

### SELECT NECESSARY COLUMNS FROM EACH SOURCES 

In [12]:
# get username and content
twitterDF_2 = twitterDF.select(col("username"),col("tweet").alias("content"))

# drop null value
twitterDF_2.na.drop("any")

# add source column
twitterDF_2 = twitterDF_2.withColumn("source", lit("Twitter"))

# show result
twitterDF_2.show()

+------------+--------------------+-------+
|    username|             content| source|
+------------+--------------------+-------+
|posindonesia|Baik sahabat, tel...|Twitter|
|posindonesia|Hai sahabat. Kiri...|Twitter|
|posindonesia|/ handphone pener...|Twitter|
|posindonesia|Hai sahabat, kiri...|Twitter|
|posindonesia|Oranger menjaga s...|Twitter|
|posindonesia|Hai sahabat. Untu...|Twitter|
|posindonesia|Terima kasih tela...|Twitter|
|posindonesia|Hai sahabat. Moho...|Twitter|
|posindonesia|Hai sahabat. Moho...|Twitter|
|posindonesia|Sama - sama sahab...|Twitter|
|posindonesia|Hai sahabat. Kiri...|Twitter|
|posindonesia|biaya pelalubeaan...|Twitter|
|posindonesia|Halo Sahabat, seb...|Twitter|
|posindonesia|Berdasarkan hasil...|Twitter|
|posindonesia|Mohon diinformasi...|Twitter|
|posindonesia|kami informasikan...|Twitter|
|posindonesia|Baik Sahabat, Moh...|Twitter|
|posindonesia|Halo Sahabat. Moh...|Twitter|
|posindonesia|Halo Sahabat pos....|Twitter|
|posindonesia|Baik sahabat moho.

In [13]:
# get username and content
hashtagDF_2 = hashtagDF.select(col("username"),col("tweet").alias("content"))

# drop null value
hashtagDF_2.na.drop("any")

# add source column
hashtagDF_2 = hashtagDF_2.withColumn("source", lit("Twitter Hashtag"))

# show result
hashtagDF_2.show()

+--------------+--------------------+---------------+
|      username|             content|         source|
+--------------+--------------------+---------------+
|  kantorpospbg|Ada Kiriman Uang?...|Twitter Hashtag|
|      detikcom|Resi Pos merupaka...|Twitter Hashtag|
|  posindonesia|Oranger, layanan ...|Twitter Hashtag|
|        k59300|#Repost posindone...|Twitter Hashtag|
|  posindonesia|Kiriman lewat Pos...|Twitter Hashtag|
|amantepatwaktu|Kenali Pos sedari...|Twitter Hashtag|
|  kantorpospbg|#Repost posindone...|Twitter Hashtag|
|  posindonesia|Mau layanan Pos I...|Twitter Hashtag|
|  posindonesia|Oranger Pos Indon...|Twitter Hashtag|
|        k59300|#Repost posindone...|Twitter Hashtag|
|     tokondutz|Sale cd audio  #s...|Twitter Hashtag|
|     tokondutz|Sale cd audio  #s...|Twitter Hashtag|
|  kantorpospbg|#Repost posindone...|Twitter Hashtag|
|  posindonesia|Hati-hati dalam m...|Twitter Hashtag|
|aboben_variasi|Kita ga cuma mela...|Twitter Hashtag|
|  therapistsby|Ready stock 

In [14]:
# create function to flatten the array
def FlatDF(schema, prefix=None):
        fields = []
        for field in schema.fields:
            name = prefix + '.' + field.name if prefix else field.name
            dtype = field.dataType
            if isinstance(dtype, ArrayType):
                dtype = dtype.elementType

            if isinstance(dtype, StructType):
                fields += FlatDF(dtype, prefix=name)
            else:
                fields.append(name)

        return fields

In [15]:
instaDF = instaDF.select(FlatDF(instaDF.schema))

In [16]:
instaDF.show()
instaDF.printSchema()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             caption|              author|             comment|            datetime|            img_urls|                 key|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Oranger menjaga s...|[mamah.asyraf, po...|[Cek DM, Baik, Sa...|2019-07-13T04:06:...|[https://scontent...|https://www.insta...|
|Oranger, layanan ...|[rizqimuhammad77,...|[@ari_hr04 @adi04...|2019-07-12T09:55:...|[https://scontent...|https://www.insta...|
|Selamat hari kope...|[evan_mardiyanto,...|[Min mohon segera...|2019-07-12T02:21:...|[https://scontent...|https://www.insta...|
|Kuliah? di @polte...|[dicky.ra_, posin...|[@posindonesia.ig...|2019-07-11T05:19:...|[https://scontent...|https://www.insta...|
|Selamat ulang tah...|[hey.rudd, jovian...|[Kak.. mau tanya....|2019-07-11T01:22:...|[https://scontent..

In [17]:
# get username and content
instaDF_2 = instaDF.withColumn("author_comment", arrays_zip("author","comment")) \
    .select(explode("author_comment").alias("ac")) \
    .select(col("ac.author").alias("username"),col("ac.comment").alias("content"))

# drop null value
instaDF_2.na.drop("any")

# add source column
instaDF_2 = instaDF_2.withColumn("source", lit("Instagram"))

# show result
instaDF_2.show()

+-------------------+--------------------+---------+
|           username|             content|   source|
+-------------------+--------------------+---------+
|       mamah.asyraf|              Cek DM|Instagram|
|    posindonesia.ig|Baik, Sahabat, mo...|Instagram|
|    akhirini_husein|Kiriman saya tida...|Instagram|
|    posindonesia.ig|Halo, Sahabat, bi...|Instagram|
|             figha_|Pengambilan paket...|Instagram|
|    posindonesia.ig|Halo, Sahabat. Mo...|Instagram|
|             figha_|@posindonesia.ig ...|Instagram|
|    posindonesia.ig|Sama-sama Sahabat...|Instagram|
|       phytastoreso|Kak tolong cek dm...|Instagram|
|    posindonesia.ig|Baik, telah kami ...|Instagram|
|             figha_|Kalo mengambil pa...|Instagram|
|    rizqimuhammad77|@ari_hr04 @adi047...|Instagram|
|griyakulakannganjuk|Kirim paket belum...|Instagram|
|             snttaa|Tolong cek STR040...|Instagram|
|           widie489|https://ecoracing...|Instagram|
|    posindonesia.ig|Boleh diinformasi...|Inst

### COMBINE ALL DATA INTO ONE TABLE

In [18]:
df1 = twitterDF_2.union(hashtagDF_2).distinct()
df2 = df1.union(instaDF_2).distinct()
df2.show()

+------------+--------------------+-------+
|    username|             content| source|
+------------+--------------------+-------+
|posindonesia|sebesar Rp20.000,...|Twitter|
|posindonesia|Halo, Sahabat. Ut...|Twitter|
|posindonesia|Baik sahabat,suda...|Twitter|
|posindonesia|POS INDONESIA (PE...|Twitter|
|posindonesia|Hai sahabat, untu...|Twitter|
|posindonesia|Ada yang dapat ka...|Twitter|
|posindonesia|Selamat ulang tah...|Twitter|
|posindonesia|Halo, sahabat. Te...|Twitter|
|posindonesia|perjalanan, mohon...|Twitter|
|posindonesia|Mohon maaf atas k...|Twitter|
|posindonesia|Mohon diinformasi...|Twitter|
|posindonesia|Utk kiriman gagal...|Twitter|
|posindonesia|Baik sahabat. moh...|Twitter|
|posindonesia|Halo, sahabat. Ki...|Twitter|
|posindonesia|Halo sahabat, Men...|Twitter|
|posindonesia|Halo sahabat, moh...|Twitter|
|posindonesia|Halo sahabat pos....|Twitter|
|posindonesia|Kiriman sahabat d...|Twitter|
|posindonesia|Halo sahabat, Moh...|Twitter|
|posindonesia|Halo sahabat Pos,.

### CLOSE SESSION

In [19]:
spark_session.stop()